## 3. 整体流程设计

本notebook对应[这一篇blog](https://hanqiu92.github.io/blogs/2020/LP_dual_simplex_solver_3_202004/)中的内容，主要包括第一阶段问题（寻找对偶可行基）以及数值稳定性控制手段的实现。

首先仍然是一些准备工作：导入相关的计算工具包、从文件util_lec_3.py中读取前两个notebook中完成的类。

In [1]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse import csc_matrix,csr_matrix,coo_matrix
from util_lec_3 import *

### 第一阶段问题（寻找对偶可行基）

在第一阶段，我们通过求解问题
$$
  \begin{aligned}
    \min \ & c^T x \\
    s.t. \ & Ax = 0; \\
           & -I(l \leq -\infty) \leq x \leq I(u \geq \infty). \\
  \end{aligned}\tag{1}
$$

来寻找初始对偶可行基。根据其形式可知，在实现中只需要改变输入$b$,$l$和$u$，然后调用DS迭代步即可。

此外，我们还需要加入对第一和第二阶段（寻找最优基）求解过程的管理。这里，我先考虑以下简单的逻辑：
1. 如果初始基对偶可行，直接进入第二阶段；
2. 否则，求解第一阶段问题，然后根据求解结果判断：
    1. 如果返回的基的状态是对偶可行的，那么继续进入第二阶段；
    2. 否则，直接退出求解过程并返回“对偶不可行”。
    
这一部分逻辑可以直接在此前留出的内部函数_solve中实现。

In [2]:
class DualSimplexSolverPlus1(DualSimplexSolver):
    '''
    在DualSimplexSolver的基础上加入第一阶段的处理
    '''
    def _solve_phase_one(self,problem,sol,basis):
        '''
        基于原始问题构造第一阶段问题（b' \gets 0, l' \gets -I(l = -\infty), u' \gets I(u = \infty)），
        然后调用DS迭代步求解
        '''
        m,n = problem.m,problem.n
        l_,u_ = np.zeros((m,)),np.zeros((m,))
        l_[problem.bool_lower_unbounded] = -100
        u_[problem.bool_upper_unbounded] = 100
        problemPhase1 = Problem(problem.A,np.zeros((n,)),problem.c,l_,u_)
        problemPhase1,solPhase1,basis = self._refactorize(problemPhase1,sol=None,basis=basis)
        statusPhase1,problemPhase1,solPhase1,basis = self._loop(problemPhase1,solPhase1,basis)
        return statusPhase1,problemPhase1,solPhase1,basis
    
    def _solve_phase_two(self,problem,sol,basis):
        problem,sol,basis = self._refactorize(problem,sol,basis)
        status,problem,sol,basis = self._loop(problem,sol,basis)
        return status,problem,sol,basis

    def _solve(self,problem,sol,basis):
        '''
        加入简单的两阶段处理逻辑
        '''
        infeas_dict,status_str = problem.check_sol_status(sol)
        if infeas_dict['unbnd']: ## 判断基是否对偶可行（sign和上下界一致）
            ## 若对偶不可行，通过第一阶段问题寻找可行基
            print('Enter phase 1.')
            statusPhase1,problemPhase1,solPhase1,basis = self._solve_phase_one(problem,sol,basis)
            ## 用第一阶段所得的基更新原始问题的解
            sol.sign = solPhase1.sign.copy()
            problem,sol,basis = self._refactorize(problem,sol,basis)
            
            ## 对第一阶段结果进行分析
            if statusPhase1 in (SolveStatus.OPT,SolveStatus.PRIMAL_INFEAS):
                ## 再次检查基是否对偶可行（或者存在原始变量无界）
                bool_unbnd = problem.eval_unbnd(sol)
                if np.any(bool_unbnd):
                    ## 对偶不可行，打印当前结果并退出
                    problem.check_sol_status(sol,print_func=print,print_header='phase 1 check ')
                    return SolveStatus.DUAL_INFEAS,problem,sol,basis
                else:
                    ## 找到初始对偶可行基，跳转到第二阶段
                    pass
            else:
                ## 第一阶段求解过程存在问题，打印当前结果并退出
                problem.check_sol_status(sol,print_func=print,print_header='phase 1 check ')
                return SolveStatus.DUAL_INFEAS,problem,sol,basis
             
        ## 求解第二阶段问题
        print('Enter phase 2.')
        status,problem,sol,basis = self._solve_phase_two(problem,sol,basis)
        return status,problem,sol,basis

下面，继续通过测试问题来评估上面第一阶段实现的效果。下面的测试函数与前一个notebook中的测试函数的主要区别在于：
* 去除了（为了保证对偶可行而引入的）虚拟上下界；
* 加入了更多的参数和求解信息输出以更好地进行比较。

In [3]:
from util import *
import time
import traceback
import glob
np.set_printoptions(suppress=False,precision=4)

def test(solve_func,max_problem_size=1e20,model_fnames=None,output_fname=None,random_seed=None):
    '''
    测试函数，各参数的定义如下：
    solve_func对应所实现的DS求解流程入口
    problem_size代表最大可容许的问题矩阵A的非零元素个数，用于约束测试问题的难度
    model_fnames对应一个测试问题路径的list，进一步提供了只对部分问题进行测试的选项
    output_fname对应结果输出路径，可输出PuLP和实现的DS流程对各测试问题的求解结果和时间，用于后续数据分析
    random_seed可指定随机种子，用于在存在随机性的求解流程（例如随机扰动）中固定随机项，保证多次调用的结果的稳定性
    '''
    if output_fname is not None:
        ## 打开输出文件并写入表头
        f = open(output_fname,'w+')
        f.write('model name\tsize\ttime solver\tobj solver\ttime pulp\tobj pulp\tobj matched?\n')
    
    if random_seed is None:
        ## 用当前时间作为随机种子
        random_seed = int(time.time())
    print('random seed = {}.'.format(random_seed))
    
    evaluator = Evaluator()
    if model_fnames is None:
        ## 默认读取所有测试问题
        model_fnames = sorted(glob.glob('netlib/*.SIF'))
    for fname in model_fnames:
        model_name = fname.split('/')[-1].split('.')[0]

        ## 读取测试问题
        A_dict,b_dict,sense_dict,c_dict,l_dict,u_dict,m,n,row_key,col_key = read_mps(fname)
        A,b,sense,c,l,u = dicts_to_computable(A_dict,b_dict,sense_dict,c_dict,l_dict,u_dict,m,n)

        ## 根据测试问题大小进行筛选
        if A.nnz < max_problem_size:
            evaluator.reset(A,b,sense,c,l,u)
            size = (A.shape,A.nnz)
            print('\nProblem name: {}, size: ({},{}).'.format(model_name,A.shape,A.nnz))
            
            time_pulp,time_ds,obj_pulp,obj_ds = np.nan,np.nan,np.nan,np.nan
            bool_match = False
            try:
                ## 调用PuLP求解
                tt = time.time()
                print("[----Launch PuLP----]")
                x_pulp,lam_pulp,status_pulp = solve_pulp(A_dict,b_dict,sense_dict,c_dict,l_dict,u_dict,
                                                         m,n,msg=1)
                time_pulp = time.time() - tt

                ## 调用实现的DS流程求解
                np.random.seed(random_seed)
                tt = time.time()
                print("[----Launch Solver----]")
                status_ds,sol_ds,basis_ds = solve_func(A,b,sense,c,l,u)
                x_ds = sol_ds.x
                time_ds = time.time() - tt
                
                ## 比较求解结果
                obj_pulp = evaluator.eval_primal_obj(x_pulp)
                obj_ds = evaluator.eval_primal_obj(x_ds)
                print("[----Begin evaluation----]")
                print(f"PuLP Eval: {evaluator.eval_str(x_pulp)} Status: {status_pulp}.")
                print(f"Solver Eval: {evaluator.eval_str(x_ds)} Status: {status_ds}.")
                print(f"Elapsed time: PuLP = {time_pulp:.3f}, Solver = {time_ds:.3f}.")
                bool_match = np.abs(obj_ds - obj_pulp) <= 1e-4 * np.abs(obj_pulp)
                print(f"Two solvers {'match' if bool_match else 'mismatch'}: "
                      f"PuLP = {obj_pulp:.4e}, Solver = {obj_ds:.4e}.")

            except Exception as e:
                print(repr(e))
                print(traceback.print_exc())
                
            if output_fname is not None:
                ## 输出求解结果
                f.write(f"{model_name}\t{size}\t{time_ds:.3f}\t{obj_ds:.4e}\t"
                        f"{time_pulp:.3f}\t{obj_pulp:.4e}\t{bool_match}\n")

    if output_fname is not None:
        f.close()

In [4]:
## 进行测试
ds_p = DualSimplexSolverPlus1()
test(ds_p.solve,max_problem_size=1000)

random seed = 1585376904.

Problem name: ADLITTLE, size: ((56, 97),383).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
6   Obj Primal -1.8677e+06 Dual -1.8677e+06  Primal Inf 4.4291e+03 (32)
phase 1 check   Obj Primal -1.8677e+20 Dual -1.8677e+20  Primal Inf 6.7291e+17 (57)  Dual Inf 1.8677e+04 (22)  Con Inf 3.0400e+01 (20)  Bnd err 23
[----Begin evaluation----]
PuLP Eval: con inf=2.6769e-05,var inf=0.0000e+00,obj=2.2549e+05. Status: 1.
Solver Eval: con inf=4.4291e+17,var inf=0.0000e+00,obj=-1.8677e+20. Status: SolveStatus.DUAL_INFEAS.
Elapsed time: PuLP = 0.109, Solver = 0.015.
Two solvers mismatch: PuLP = 2.2549e+05, Solver = -1.8677e+20.

Problem name: AFIRO, size: ((27, 32),83).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
5   Obj Primal -1.7774e+02 Dual -1.7774e+02  Primal Inf 7.9960e+02 (12)
phase 1 check   Obj Primal -1.7774e+16 Dual -1.7774e+16  Primal Inf 1.1996e+17 (16)  Dual Inf 1.7774e+00 (4)  Bnd err 4
[----Begin evaluation----]
PuLP Eval: co

/Users/hanqiu/dual_simplex_solve/util_lec_3.py:515: RuntimeWarning: divide by zero encountered in double_scalars
  alpha_primal = (-primal_gap - delta_xBI) / xB_grad[idxI]
/Users/hanqiu/dual_simplex_solve/util_lec_3.py:516: RuntimeWarning: invalid value encountered in multiply
  sol.x[basis.idxB] += alpha_primal * xB_grad
/Users/hanqiu/dual_simplex_solve/util_lec_3.py:528: RuntimeWarning: divide by zero encountered in true_divide
  eta_vec = -xB_grad0 / xB_grad0[idxI]
/Users/hanqiu/dual_simplex_solve/util_lec_3.py:528: RuntimeWarning: invalid value encountered in true_divide
  eta_vec = -xB_grad0 / xB_grad0[idxI]
/Users/hanqiu/dual_simplex_solve/util_lec_3.py:529: RuntimeWarning: divide by zero encountered in double_scalars
  eta_vec[idxI] += 1 / xB_grad0[idxI]
/Users/hanqiu/dual_simplex_solve/util_lec_3.py:352: RuntimeWarning: divide by zero encountered in double_scalars
  betaI = betaI0 / alpha_j / alpha_j
/Users/hanqiu/dual_simplex_solve/util_lec_3.py:353: RuntimeWarning: invalid va

Enter phase 1.
32   Obj Primal -3.9199e-15 Dual 0.0000e+00
Enter phase 2.
54   Obj Primal -6.4575e+01 Dual -6.4575e+01
[----Begin evaluation----]
PuLP Eval: con inf=3.7800e-05,var inf=0.0000e+00,obj=-6.4575e+01. Status: 1.
Solver Eval: con inf=1.3034e-12,var inf=0.0000e+00,obj=-6.4575e+01. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.035, Solver = 0.045.
Two solvers match: PuLP = -6.4575e+01, Solver = -6.4575e+01.

Problem name: SC50B, size: ((50, 48),118).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
18   Obj Primal -3.3729e+01 Dual -3.3729e+01  Primal Inf 3.5185e+02 (9)
phase 1 check   Obj Primal -3.3729e+15 Dual -3.3729e+15  Primal Inf 5.5185e+16 (11)  Dual Inf 3.3729e-01 (2)  Con Inf 2.5000e+00 (5)  Bnd err 2
[----Begin evaluation----]
PuLP Eval: con inf=6.3949e-14,var inf=0.0000e+00,obj=-7.0000e+01. Status: 1.
Solver Eval: con inf=3.5185e+16,var inf=0.0000e+00,obj=-3.3729e+15. Status: SolveStatus.DUAL_INFEAS.
Elapsed time: PuLP = 0.045, Solver = 0.018.
Two solve

上面的求解结果显示，目前实现的两阶段DS过程对于大多数问题并不work。特别地，根据phase 1 check输出结果可以看出，许多测试问题的第一阶段问题并不能被正确求解，导致求解过程过早退出。造成这种现象的一部分（主要）原因是DS迭代过程_loop没有对数值误差进行有效处理；下面，我将实现一些解决方案。

### Robust Ratio Test

在这一小节中，我将实现robust ratio test。blog中已经介绍了该方法的基本思路，这里简单地回顾一下步骤：
1. 计算各列$j$的临界对偶步长$\alpha_j$和区间$[\alpha_j^-,\alpha_j^+]$；
2. 用二分法确定对偶步长上界$\alpha_{dual}^+$和对应的列$j^+$；
3. 找到所有$\alpha_j$落在区间$[\alpha_{j^+},\alpha_{dual}^+)$中的列$j$，并选择$|\delta s_j|$最大的一个。

下面是具体的代码实现。

In [5]:
def ratio_test(problem,sol,basis,s_grad,dual_grad):
    '''
    ratio test. 
    
    输入
    problem,sol,basis: 问题、解和基
    s_grad: s的单位变化量
    dual_grad: 对偶目标的单位变化量（即对偶梯度）
    
    输出
    status: 求解状态
    idxJ: 选择的列在N=L\cup U中的位置
    idxNJ: 选择的列在\{1,\cdots,m\}中的位置
    alpha_dual: 对偶步长
    flip_list: 需要翻转类型的列的list
    check_list: 有可能对偶不可行、需要做shift的列的list
    '''
    idxN,boolN = basis.idxN,basis.boolN

    ## 统计可能会约束对偶步长的对偶变量的下标/列
    idxL_bounded = np.where((sol.sign == VarStatus.AT_LOWER_BOUND.value) & (s_grad < -ZERO_TOL))[0]## 处于下界，要求s>=0
    idxU_bounded = np.where((sol.sign == VarStatus.AT_UPPER_BOUND.value) & (s_grad > ZERO_TOL))[0]## 处于上界，要求s<=0
    idxF = np.where((sol.sign == VarStatus.OTHER.value) & boolN)[0]## free变量，要求s==0
    idxF_bounded = idxF[(np.abs(s_grad[idxF]) > ZERO_TOL)]
    elems_bounded = np.concatenate([idxL_bounded,idxU_bounded,idxF_bounded])

    if len(elems_bounded) == 0:
        ## 没有列可以约束对偶步长，因此对偶步长可以无限大，从而对偶目标无界/原始解不可行
        status,idxJ,idxNJ,alpha_dual,flip_list,check_list = SolveStatus.PRIMAL_INFEAS,-1,-1,0,[],[]
        return status,idxJ,idxNJ,alpha_dual,flip_list,check_list

    ## 针对可能约束对偶步长的列，进一步判断其是否可以做bound flip；如果可行，则进行相关计算
    bool_not_both_bounded = problem.bool_not_both_bounded[elems_bounded]
    s_grad_bounded = s_grad[elems_bounded]
    ## 计算bound filp对对偶梯度的影响
    s_grad_abs_bounded = np.abs(s_grad_bounded)
    dual_grad_delta_flipped = problem.bounds_gap[elems_bounded] * s_grad_abs_bounded
    if (np.sum(dual_grad_delta_flipped) <= dual_grad - DUAL_TOL) and (not np.any(bool_not_both_bounded)):
        ## 如果所有约束列都可以做bound flip，而且flip完对偶的梯度仍是正数，则对偶目标无界/原始解不可行
        status,idxJ,idxNJ,alpha_dual,flip_list,check_list = SolveStatus.PRIMAL_INFEAS,-1,-1,0,[],[]
        return status,idxJ,idxNJ,alpha_dual,flip_list,check_list

    ## step 1: 计算每个约束变量对应bound flip的临界对偶步长
    alpha_dual_allowed = - sol.s[elems_bounded] / s_grad_bounded
    alpha_dual_allowed_ub = alpha_dual_allowed + DUAL_TOL / s_grad_abs_bounded

    ## step 2: 通过二分法确定对偶步长上界alpha_dual_ub和对应的列idxs_pivot_ub
    ## 首先，确定alpha_dual_ub搜索的上界alpha_dual_ub_ub和下界alpha_dual_ub_lb
    idxs_pivot_ub = []
    if np.any(bool_not_both_bounded):
        ## 存在不可flip的列，可以大幅降低搜索上界
        alpha_dual_ub_ub = np.min(alpha_dual_allowed_ub[bool_not_both_bounded])
        ## 找到对应最小步长的不可flip列之前的所有列
        idxs_remain = np.where(alpha_dual_allowed_ub < alpha_dual_ub_ub)[0]
        if len(idxs_remain) == 0:
            ## 如果idxs_remain为空，可以直接确定alpha_dual_ub
            alpha_dual_ub = alpha_dual_ub_lb = alpha_dual_ub_ub
            idxs_pivot_ub = list(np.where(alpha_dual_allowed_ub == alpha_dual_ub)[0])
        elif np.sum(dual_grad_delta_flipped[idxs_remain]) <= dual_grad - DUAL_TOL:
            ## 如果idxs_remain全部flip完对偶的梯度仍是正数，可以直接确定alpha_dual_ub
            alpha_dual_ub = alpha_dual_ub_lb = alpha_dual_ub_ub
            idxs_pivot_ub = list(np.where(alpha_dual_allowed_ub == alpha_dual_ub)[0])
        else:
            ## 否则需要进一步搜索
            alpha_dual_ub_lb = np.min(alpha_dual_allowed_ub)
            alpha_dual_ub_ub = np.max(alpha_dual_allowed_ub[idxs_remain])
    else:
        ## 考虑全部列
        alpha_dual_ub_lb = np.min(alpha_dual_allowed_ub)
        alpha_dual_ub_ub = np.max(alpha_dual_allowed_ub)
        idxs_remain = np.arange(len(elems_bounded),dtype=int)

    if len(idxs_pivot_ub) == 0:
        ## 根据区间[alpha_dual_ub_lb,alpha_dual_ub_ub]做二分查找
        dual_grad_tmp = dual_grad - DUAL_TOL
        alpha_dual_ub = (alpha_dual_ub_lb + alpha_dual_ub_ub) / 2
        while len(idxs_remain) > 2 and dual_grad_tmp >= 0 and (alpha_dual_ub_ub - alpha_dual_ub_lb) > 2 * DUAL_TOL / PIVOT_TOL:
            bool_selected = alpha_dual_allowed_ub[idxs_remain] <= alpha_dual_ub
            if not np.any(bool_selected):
                ## 没有覆盖任何列，直接扩大下界
                alpha_dual_ub_lb = alpha_dual_ub
                alpha_dual_ub = (alpha_dual_ub_lb + alpha_dual_ub_ub) / 2
                continue

            ## 计算dual_grad的变化量
            idxs_selected = idxs_remain[bool_selected]
            delta_dual_grad = np.sum(dual_grad_delta_flipped[idxs_selected])
            if (dual_grad_tmp < delta_dual_grad):
                ## 超过dual_grad，降低上界
                idxs_remain = idxs_selected
                alpha_dual_ub_ub = alpha_dual_ub
            else:
                ## 不超过dual_grad，扩大下界
                dual_grad_tmp -= delta_dual_grad
                idxs_remain = idxs_remain[~bool_selected]
                alpha_dual_ub_lb = alpha_dual_ub
            alpha_dual_ub = (alpha_dual_ub_lb + alpha_dual_ub_ub) / 2

        if len(idxs_remain) > 1:
            ## 如果二分查找留下多个列未处理，很可能是一个小区间[alpha_dual_ub_lb,alpha_dual_ub_ub]中包含多个列，直接排序搜索
            idxs_remain = idxs_remain[np.argsort(alpha_dual_allowed_ub[idxs_remain])]
            delta_dual_grads = np.cumsum(dual_grad_delta_flipped[idxs_remain])
            idx_first_exceeding = int(np.sum(delta_dual_grads <= dual_grad_tmp)) ## 找到首个超过dual_grad的列
            if idx_first_exceeding < len(idxs_remain):
                idxs_pivot_ub = [idxs_remain[idx_first_exceeding]]
            else:
                idxs_pivot_ub = [idxs_remain[-1]]
            alpha_dual_ub = alpha_dual_allowed_ub[idxs_pivot_ub[0]]
        elif len(idxs_remain) == 1:
            ## 如果二分查找后只留下一个列未处理，那么选取该列和对应的对偶步长作为上界
            idxs_pivot_ub = list(idxs_remain)
            alpha_dual_ub = alpha_dual_allowed_ub[idxs_pivot_ub[0]]
        else:
            ## 理论上二分查找后至少有一个列未处理；报错，并当做primal inf退出
            print('error! length of remain idxs = 0.')
            status,idxJ,idxNJ,alpha_dual,flip_list,check_list = SolveStatus.PRIMAL_INFEAS,-1,-1,0,[],[]
            return status,idxJ,idxNJ,alpha_dual,flip_list,check_list

    ## step 3: 根据alpha_dual_ub确定区间，找出所有应考虑的列，并从中选出|s_grad|最大的一个
    alpha_dual_lb = min(alpha_dual_allowed[idxs_pivot_ub])
    idxs_pivot = idxs_pivot_ub
    idxs_selected = np.where((alpha_dual_allowed < alpha_dual_ub) & (alpha_dual_allowed >= alpha_dual_lb))[0]
    if len(idxs_selected) > 0:
        idxs_selected = list(set(list(idxs_selected) + idxs_pivot))
    else:
        idxs_selected = idxs_pivot
    idx_pivot = idxs_selected[np.argmax(s_grad_abs_bounded[idxs_selected])]
    idxNJ = elems_bounded[idx_pivot]
    alpha_dual = alpha_dual_allowed[idx_pivot]

    ## step 4: 进行一些后处理
    if s_grad_abs_bounded[idx_pivot] <= PIVOT_TOL:
        if s_grad_abs_bounded[idx_pivot] <= DUAL_TOL:
            ## pivot size过小，直接进入回滚流程（下面进一步实现）
            status,idxJ,idxNJ,alpha_dual,flip_list,check_list = SolveStatus.ROLLBACK,-1,-1,0,[],[]
            return status,idxJ,idxNJ,alpha_dual,flip_list,check_list

    bool_flip = (alpha_dual_allowed_ub <= alpha_dual)
    flip_list = elems_bounded[bool_flip]
    alpha_dual = max(alpha_dual,ZERO_TOL) ## 扩大对偶步长至某个阈值
    idxJ = np.where(idxN == idxNJ)[0]
    ## 获取需要检查对偶可行性的列
    check_list = elems_bounded[alpha_dual_allowed_ub <= alpha_dual]
    if idxNJ not in check_list:
        check_list = np.array(list(check_list) + [idxNJ])
    return SolveStatus.ONGOING,idxJ,idxNJ,alpha_dual,flip_list,check_list

class DualSimplexSolverPlus2(DualSimplexSolverPlus1):
    '''
    在DualSimplexSolverPlus1的基础上加入robust ratio test
    在这一版中暂时不考虑shift
    '''
    
    def _ratio_test(self,problem,sol,basis,s_grad,dual_grad):
        ## 去除返回值中的check list，以保证_step对_ratio_test调用的返回值是正确的
        return ratio_test(problem,sol,basis,s_grad,dual_grad)[:-1]

下面通过测试来看一下robust ratio test的效果。

In [6]:
ds_p = DualSimplexSolverPlus2()
# test(ds_p.solve,1000)
test(ds_p.solve,10000) ## 扩大问题尺寸，进行更多测试

random seed = 1585377012.

Problem name: ADLITTLE, size: ((56, 97),383).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
24   Obj Primal -1.1344e-09 Dual 0.0000e+00
Enter phase 2.
110   Obj Primal 2.2549e+05 Dual 2.2549e+05
[----Begin evaluation----]
PuLP Eval: con inf=2.6769e-05,var inf=0.0000e+00,obj=2.2549e+05. Status: 1.
Solver Eval: con inf=1.5539e-10,var inf=0.0000e+00,obj=2.2549e+05. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.020, Solver = 0.069.
Two solvers match: PuLP = 2.2549e+05, Solver = 2.2549e+05.

Problem name: AFIRO, size: ((27, 32),83).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
11   Obj Primal -1.1369e-14 Dual 0.0000e+00
Enter phase 2.
19   Obj Primal -4.6475e+02 Dual -4.6475e+02
[----Begin evaluation----]
PuLP Eval: con inf=9.5400e-06,var inf=0.0000e+00,obj=-4.6475e+02. Status: 1.
Solver Eval: con inf=1.3500e-13,var inf=0.0000e+00,obj=-4.6475e+02. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.014, Solver = 0.016.
Two solvers matc


Problem name: DEGEN2, size: ((444, 534),3978).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
116   Obj Primal 1.6904e-12 Dual 0.0000e+00
Enter phase 2.
612   Obj Primal -1.4352e+03 Dual -1.4352e+03
[----Begin evaluation----]
PuLP Eval: con inf=1.1102e-16,var inf=0.0000e+00,obj=-1.4352e+03. Status: 1.
Solver Eval: con inf=9.3961e-12,var inf=3.2416e-12,obj=-1.4352e+03. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.072, Solver = 0.400.
Two solvers match: PuLP = -1.4352e+03, Solver = -1.4352e+03.

Problem name: E226, size: ((223, 282),2578).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
168   Obj Primal -7.0900e-14 Dual 0.0000e+00
Enter phase 2.
371   Obj Primal -1.8752e+01 Dual -1.8752e+01
[----Begin evaluation----]
PuLP Eval: con inf=9.3338e-06,var inf=0.0000e+00,obj=-1.8752e+01. Status: 1.
Solver Eval: con inf=3.0661e-10,var inf=7.0406e-13,obj=-1.8752e+01. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.044, Solver = 0.222.
Two solvers match: PuLP = -1.8


Problem name: FIT1P, size: ((627, 1677),9868).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 2.
916   Obj Primal 9.1464e+03 Dual 9.1464e+03
[----Begin evaluation----]
PuLP Eval: con inf=1.9123e-04,var inf=0.0000e+00,obj=9.1464e+03. Status: 1.
Solver Eval: con inf=3.6994e-08,var inf=0.0000e+00,obj=9.1464e+03. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.185, Solver = 0.965.
Two solvers match: PuLP = 9.1464e+03, Solver = 9.1464e+03.

Problem name: FORPLAN, size: ((161, 422),4564).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
36   Obj Primal -1.9852e-11 Dual 0.0000e+00
Enter phase 2.
275   Obj Primal -6.6422e+02 Dual -6.6422e+02
[----Begin evaluation----]
PuLP Eval: con inf=6.0633e-03,var inf=0.0000e+00,obj=-6.6422e+02. Status: 1.
Solver Eval: con inf=4.9643e-06,var inf=2.4401e-09,obj=-6.6422e+02. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.057, Solver = 0.142.
Two solvers match: PuLP = -6.6422e+02, Solver = -6.6422e+02.

Problem name: GANGES, size: ((1


Problem name: GROW7, size: ((140, 301),2612).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 2.
313   Obj Primal -4.7788e+07 Dual -4.7788e+07
[----Begin evaluation----]
PuLP Eval: con inf=2.1059e-01,var inf=1.4729e-10,obj=-4.7788e+07. Status: 1.
Solver Eval: con inf=3.8777e-08,var inf=0.0000e+00,obj=-4.7788e+07. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.045, Solver = 0.277.
Two solvers match: PuLP = -4.7788e+07, Solver = -4.7788e+07.

Problem name: ISRAEL, size: ((174, 142),2269).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
139   Obj Primal 2.8874e-09 Dual 0.0000e+00
Enter phase 2.
323   Obj Primal -8.9664e+05 Dual -8.9664e+05
[----Begin evaluation----]
PuLP Eval: con inf=9.6412e-03,var inf=0.0000e+00,obj=-8.9664e+05. Status: 1.
Solver Eval: con inf=1.4621e-05,var inf=0.0000e+00,obj=-8.9664e+05. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.044, Solver = 0.221.
Two solvers match: PuLP = -8.9664e+05, Solver = -8.9664e+05.

Problem name: KB2, size: ((

896   WARN err DSE accuracy -4.6243e-01.
897   WARN err DSE accuracy 1.4715e+01.
898   WARN err DSE accuracy -5.5183e-02.
899   WARN err DSE accuracy -4.3338e-03.
906   WARN err DSE accuracy -1.3069e-04.
907   WARN err DSE accuracy 5.7327e-02.
909   WARN err DSE accuracy 2.2514e-04.
910   WARN err DSE accuracy -5.2540e-01.
911   WARN err DSE accuracy -5.1383e-04.
919   WARN err DSE accuracy 1.2835e+00.
936   WARN err DSE accuracy -3.3240e-01.
939   WARN err DSE accuracy -1.9099e-03.
943   WARN err DSE accuracy -2.1094e-04.
948   WARN err DSE accuracy 2.5216e+01.
949   WARN err DSE accuracy 1.2908e+02.
951   WARN err DSE accuracy 3.7648e-02.
961   WARN err DSE accuracy -8.1119e+01.
962   WARN err DSE accuracy -9.0493e-02.
965   WARN err DSE accuracy -1.9073e-04.
970   WARN err DSE accuracy 1.8094e-04.
972   WARN err DSE accuracy 2.7849e-01.
975   WARN err DSE accuracy 1.0120e-04.
983   WARN err DSE accuracy 1.4363e-02.
986   WARN err DSE accuracy -2.0222e-01.
990   WARN err DSE accuracy

1578   WARN err DSE accuracy -3.9049e-01.
1580   WARN err DSE accuracy -6.0547e-02.
1581   WARN err DSE accuracy 3.6085e-04.
1582   WARN err DSE accuracy -9.3579e-04.
1583   WARN err DSE accuracy -1.0195e-03.
1584   WARN err DSE accuracy 1.0729e-03.
1585   WARN err DSE accuracy -7.9823e-04.
1586   WARN err DSE accuracy 1.4377e-04.
1587   WARN err DSE accuracy -1.4629e-03.
1592   WARN err DSE accuracy 5.1327e-04.
1593   WARN err DSE accuracy -1.8512e-03.
1594   WARN err DSE accuracy 4.4568e-03.
1596   WARN err DSE accuracy -7.7462e-03.
1597   WARN err DSE accuracy -2.6783e-03.
1600   WARN err DSE accuracy -3.9210e-04.
1603   WARN err DSE accuracy -4.7827e-04.
1605   WARN err DSE accuracy 1.6609e-03.
1608   WARN err DSE accuracy 6.3019e-02.
1617   WARN err DSE accuracy 7.5912e-04.
1620   WARN err DSE accuracy -2.2762e-02.
1629   WARN err DSE accuracy -5.2491e+00.
1632   WARN err DSE accuracy -1.3233e-02.
1633   WARN err DSE accuracy 1.6403e-04.
1635   WARN err DSE accuracy -3.0461e-04.
1

/Users/hanqiu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in less
/Users/hanqiu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in greater



Problem name: PILOT4, size: ((410, 1000),5141).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
110   Obj Primal 1.4488e-14 Dual 0.0000e+00
Enter phase 2.
422   WARN err DSE accuracy -4.5251e-04.
468   WARN err DSE accuracy 2.6727e-04.
511   WARN err DSE accuracy -4.2500e-02.
513   WARN err DSE accuracy -1.9774e-03.
514   WARN err DSE accuracy -1.2677e-03.
552   WARN err DSE accuracy -7.4327e-04.
564   WARN err DSE accuracy -4.2506e-02.
565   WARN err DSE accuracy -2.2340e-04.
568   WARN err DSE accuracy -4.9400e-04.
570   WARN err DSE accuracy 1.0109e-04.
614   WARN err DSE accuracy 5.1270e-03.
633   WARN err DSE accuracy 2.1696e-04.
642   WARN err DSE accuracy 9.2762e-04.
648   WARN err DSE accuracy 5.1338e-02.
689   WARN err DSE accuracy 3.4136e-03.
698   WARN err DSE accuracy -1.3428e-03.
730   Obj Primal -2.5811e+03 Dual -2.5811e+03
[----Begin evaluation----]
PuLP Eval: con inf=8.6423e-02,var inf=1.5930e-03,obj=-2.5811e+03. Status: 1.
Solver Eval: con inf=6.6747e-07,


Problem name: SCSD6, size: ((147, 1350),4316).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 2.
261   Obj Primal 5.0500e+01 Dual 5.0500e+01
[----Begin evaluation----]
PuLP Eval: con inf=1.0431e-07,var inf=0.0000e+00,obj=5.0500e+01. Status: 1.
Solver Eval: con inf=2.8394e-13,var inf=5.0147e-13,obj=5.0500e+01. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.103, Solver = 0.157.
Two solvers match: PuLP = 5.0500e+01, Solver = 5.0500e+01.

Problem name: SCSD8, size: ((397, 2750),8584).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 2.
961   Obj Primal 9.0500e+02 Dual 9.0500e+02
[----Begin evaluation----]
PuLP Eval: con inf=5.8315e-06,var inf=0.0000e+00,obj=9.0500e+02. Status: 1.
Solver Eval: con inf=3.3944e-11,var inf=6.2166e-12,obj=9.0500e+02. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.282, Solver = 0.706.
Two solvers match: PuLP = 9.0500e+02, Solver = 9.0500e+02.

Problem name: SCTAP1, size: ((300, 480),1692).
[----Launch PuLP----]
[----Launch Solver----]
Ente


Problem name: STOCFOR2, size: ((2157, 2031),8343).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
1000   Obj Primal -1.2241e+05 Dual -1.2241e+05  Primal Inf 1.0535e+05 (676)
1901   Obj Primal 7.1942e-10 Dual 0.0000e+00
Enter phase 2.
2000   Obj Primal -4.0672e+04 Dual -4.0672e+04  Primal Inf 1.4978e+03 (151)
2119   Obj Primal -3.9024e+04 Dual -3.9024e+04
[----Begin evaluation----]
PuLP Eval: con inf=1.0366e-02,var inf=0.0000e+00,obj=-3.9024e+04. Status: 1.
Solver Eval: con inf=1.6708e-09,var inf=1.0152e-12,obj=-3.9024e+04. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.269, Solver = 3.009.
Two solvers match: PuLP = -3.9024e+04, Solver = -3.9024e+04.

Problem name: TUFF, size: ((333, 587),4520).
[----Launch PuLP----]
[----Launch Solver----]
Enter phase 1.
6   Obj Primal 0.0000e+00 Dual 0.0000e+00
Enter phase 2.
275   Obj Primal 2.9215e-01 Dual 2.9215e-01
[----Begin evaluation----]
PuLP Eval: con inf=2.9937e-04,var inf=0.0000e+00,obj=2.9215e-01. Status: 1.
Solver Eval: co

上面的测试结果显示，加入robust ratio test后，大部分测试问题可以被正确求解，但仍存在以下问题：
* 对于一些测试问题（包括PEROLD、PILOT-WE），robust ratio test无法避免选中对应小的pivot size的列；
* 对于一些测试问题（包括FINNIS、LOTFI、MAROS），在第一和第二阶段的转换中会发生可行性丧失的情况（见phase 1 check输出）。

在下一小节中，我将进一步引入随机扰动和回滚机制，并对求解主流程的逻辑进行梳理和改进，以解决上述问题。

### 随机扰动-回滚机制与求解流程改进

首先，我将在DualSimplexSolver中加入随机扰动和shift；后续的回滚机制和求解流程控制都需要使用这两种技术。

In [7]:
class DualSimplexSolverPlus3a(DualSimplexSolverPlus2):
    '''
    在DualSimplexSolverPlus2的基础上加入shift和perturb技术
    '''
    
    def _get_header(self):
        count = self.global_info.get('count',0)
        phase = self.global_info.get('phase',2)
        header = '{} (P{})'.format(count,phase)
        return header
    
    def _perturb(self,problem):
        '''
        针对DS算法的随机扰动方法，参考A. Koberstein论文中的实现
        '''
        ## 先获取原始目标系数
        c = self.global_info['c_raw'].copy()
    
        ## 获取问题属性
        l,u = problem.l,problem.u
        n,m = problem.A.shape
        m_original = m - n

        ## 生成随机扰动的大小
        psi = 1e-5
        perturb_scale = psi * np.abs(c) + 1e2 * DUAL_TOL
        perturb_scale = 0.5 * (1 + np.random.rand(len(c))) * perturb_scale
        ## 根据上下界情况，基于对偶可行性调整扰动方向
        perturb_scale = (1 * (u >= INF) - 1 * (l <= -INF) + 1 * ((u < INF) & (l > -INF))) * perturb_scale

        ## 将扰动值缩放到范围[perturb_min,perturb_max]中
        perturb_min,perturb_max = min(psi,1e-1 * DUAL_TOL),max(psi*np.mean(c),1e2 * DUAL_TOL)
        idx_below = (np.abs(perturb_scale) < perturb_min) & (perturb_scale != 0)
        while np.any(idx_below):
            perturb_scale[idx_below] *= 10
            idx_below = (np.abs(perturb_scale) < perturb_min) & (perturb_scale != 0)
        idx_above = (np.abs(perturb_scale) > perturb_max)
        while np.any(idx_above):
            perturb_scale[idx_above] /= 10
            idx_above = (np.abs(perturb_scale) > perturb_max)

        ## 施加扰动，并保证逻辑变量不受影响
        c_perturb = c + perturb_scale
        c_perturb[m_original:] = 0

        ## 生成扰动后的问题
        problem_perturb = Problem(problem.A,problem.b,c_perturb,problem.l,problem.u)
        
        ## 打印扰动量级
        header = self._get_header()
        print('{}  INFO max perturb size {:.2e}.'.format(header,np.max(np.abs(c_perturb - c))))
        return problem_perturb
    
    def _shift(self,problem,sol,check_list=None):
        '''
        对目标系数c进行shift调整，使得对偶解sol.s是可行的；
        参数check_list指定了需要考虑shift的列
        '''
        if check_list is not None:
            s_,sign_ = sol.s[check_list],sol.sign[check_list]
        else:
            s_,sign_ = sol.s,sol.sign
            
        ## 计算解的对偶不可行程度
        bool_lower = sign_ == VarStatus.AT_LOWER_BOUND.value
        bool_upper = sign_ == VarStatus.AT_UPPER_BOUND.value
        dual_inf_ = np.zeros((len(s_),))
        dual_inf_[bool_upper] = np.maximum(s_[bool_upper],0)
        dual_inf_[bool_lower] = np.maximum(-s_[bool_lower],0)
            
        bool_shift = dual_inf_ > DUAL_TOL
        if np.any(bool_shift):
            ## 找到对偶不可行的列，计算应该shift的值
            shift_ = s_[bool_shift]
            shift_[bool_lower[bool_shift]] += DUAL_TOL
            shift_[bool_upper[bool_shift]] -= DUAL_TOL
                
            ## 做shift
            if check_list is not None:
                problem.c[check_list[bool_shift]] -= shift_
                sol.s[check_list[bool_shift]] -= shift_
            else:
                problem.c[bool_shift] -= shift_
                sol.s[bool_shift] -= shift_
                
            ## 如有必要，打印shift量级
            if np.max(np.abs(shift_)) > DUAL_TOL * 10:
                header = self._get_header()
                print('{}  INFO max shift size: {:.2e}.'.format(header,np.max(np.abs(shift_))))
        return problem,sol

接下来，我将介绍回滚机制的设计。

根据blog中的思路，在迭代流程无法正常继续下去时应触发回滚操作，回到迭代路径上一个之前的好的基/解，施加随机扰动，然后重新开始求解。为此，我们应在迭代过程中定期保留好的解，以供后续可能的回滚操作使用。在本系列的实现中，我引入了一个堆栈结构对这些回滚option进行维护：一方面，迭代过程定期将满足条件的解压入该堆栈中；另一方面，如果触发了回滚操作，则从堆栈中选取迭代路径上最近的（即迭代次数最大的）一个基/解来重新开始求解。

在回滚机制的具体实现中，还需要注意到一个小问题：回滚前后的求解阶段可能不一致，因此在堆栈中需要加入对每个基/解所处的求解阶段的记录，并在触发回滚后对求解阶段的一致性进行判断，及时调整求解流程。

下面的代码块在DualSimplexSolver中加入了回滚机制的基本元素，其中_save_recent函数负责将解加入堆栈中，而_rollback函数负责执行回滚操作（从堆栈中取出解、进行扰动、检查求解阶段）。

In [8]:
class DualSimplexSolverPlus3b(DualSimplexSolverPlus3a):
    '''
    在DualSimplexSolverPlus3a的基础上加入rollback机制
    '''
    
    def _save_recent(self,problem,sol,basis,count,phase):
        '''
        将当前解加入回滚堆栈中
        '''
        self.global_info['rollback_stack'] += [(count,phase,basis.copy(),sol.copy())]
            
    def _rollback(self,problem,phase):
        '''
        执行回滚操作
        '''
        ## 回滚到堆栈中的最近一个解
        if len(self.global_info['rollback_stack']) > 1:
            last_saved_sol = self.global_info['rollback_stack'].pop()
        else:
            last_saved_sol = self.global_info['rollback_stack'][0]
        last_count,last_phase,basis,sol = last_saved_sol
        header = self._get_header()
        print('{}  INFO rollback to iter {} (P{}).'.format(header,last_count,last_phase))
        basis.lu_factorize() ## 对基进行重分解
        
        ## 重新对问题进行扰动
        problem = self._perturb(problem)
        ## 检查当前解对扰动后的问题的对偶可行性，并适当做shift
        problem,sol = self._shift(problem,sol)

        ## 检查回滚前后求解阶段是否一致
        if phase != last_phase:
            ## 求解阶段不一致，则需要跳出当前DS迭代流程
            if last_phase == 1:
                status = SolveStatus.PHASE1
            elif last_phase == 2:
                status = SolveStatus.PHASE2
        else:
            ## 求解阶段一致，可以继续求解
            status = SolveStatus.ONGOING
        return status,problem,sol,basis

最后一步是在DualSimplexSolver中加入上述机制，并对求解流程进行改进。具体来说，我将：
* 在迭代步_step中增加数值误差检验和shift操作；
* 在迭代过程_loop中加入对状态REFACTOR和ROLLBACK的处理，以及通过调用_save_recent函数将好的解压入堆栈中；
* 在求解过程_solve中对第一与第二阶段解的切换步骤进行改进，以提高求解过程的鲁棒性。

具体的改动点可参见下面代码块中的注释。

In [9]:
class DualSimplexSolverPlus3(DualSimplexSolverPlus3b):
    '''
    在DualSimplexSolverPlus3b的基础上改进求解流程
    '''
    
    def _ratio_test(self,problem,sol,basis,s_grad,dual_grad):
        return ratio_test(problem,sol,basis,s_grad,dual_grad)
    
    def _step(self,problem,sol,basis):
        '''
        增加shift和数值误差检验
        '''
        count = self.global_info.get('count',0)
        phase = self.global_info.get('phase',2) ## 增加对phase的记录
        header = '{} (P{})'.format(count,phase)

        ## step 1: pricing, 选出离开下标idxBI = idxB[idxI], 并计算相应对偶变量的单位变化量
        status_inner,idxI,idxBI,primal_gap = self._pricing(problem,sol,basis)
        if status_inner != SolveStatus.ONGOING:
            return status_inner,problem,sol,basis
        dual_grad = abs(primal_gap) ## 原始变量的不可行程度正是对偶问题的梯度

        bool_to_lower_bound = sol.x[idxBI] <= problem.l[idxBI]
        direcDualI = 1 if bool_to_lower_bound else -1 ## 原始变量的移动方向
        
        ## 计算对偶变量的单位变化量
        sB_grad0 = basis.get_elem_vec(idxI,if_transpose=True) ## A_B^{-T}e_I
        lam_grad0 = basis.solve(sB_grad0,if_transpose=True) ## A_B^{-T}e_I
        s_grad0 = basis.dot(lam_grad0,if_transpose=True) ## A^TA_B^{-T}e_I
        if direcDualI == -1:
            lam_grad = lam_grad0
            s_grad = -s_grad0
        else:
            lam_grad = -lam_grad0
            s_grad = s_grad0

        ## step 2: ratio test, 选出进入下标idxNJ = idxN[idxJ]
        status_inner,idxJ,idxNJ,alpha_dual,flip_list,check_list = self._ratio_test(problem,sol,basis,s_grad,dual_grad)
        if status_inner != SolveStatus.ONGOING:
            return status_inner,problem,sol,basis

        ## step 3: 更新结果
        aNJ = basis.get_col(idxNJ) ## A_j
        xB_grad0 = basis.solve(aNJ,if_transpose=False) ## A_B^{-1}A_j
        xB_grad = - xB_grad0
        betaI = np.dot(lam_grad0,lam_grad0)
        tau = basis.solve(lam_grad0,if_transpose=False)
        
        ## 校核数值稳定性并进行处理
        if True:
            ## 校核通过\delta s和\delta x_B计算得到的alpha = e_I^T A_B^{-1} a_{NJ}的一致性
            err_pivot = s_grad0[idxNJ] + xB_grad[idxI]
            if abs(err_pivot) > PRIMAL_TOL * (1 + abs(xB_grad[idxI])):
                print('{}  WARN FTRAN/BTRAN pivot consistency err {:.4e}.'.format(header,err_pivot))
                return SolveStatus.REFACTOR,problem,sol,basis
            ## 校核通过\delta s计算得到的e_I^T A_B^T A_B^{-T} e_I = e_I^T e_I = 1的准确性
            err_btran = s_grad0[idxBI] - 1
            if abs(err_btran) > DUAL_TOL:
                print('{}  WARN BTRAN accuracy err {:.4e}.'.format(header,err_btran))
                return SolveStatus.REFACTOR,problem,sol,basis
            ## 校核pivot element的大小
            if abs(xB_grad[idxI]) < PIVOT_TOL:
                print('{}  WARN pivot size {:.4e}.'.format(header,xB_grad[idxI]))
                if abs(xB_grad[idxI]) < ZERO_TOL:
                    return SolveStatus.ROLLBACK,problem,sol,basis
        if False:
            ## 校核DSE权重的准确性
            err_dse = betaI - basis.DSE_weights[idxI]
            if abs(err_dse) > PIVOT_TOL * 10:
                print('{}  WARN DSE accuracy err {:.4e}.'.format(header,err_dse))
        
        ## 更新对偶变量
        sol.lam += alpha_dual * lam_grad
        sol.s += alpha_dual * s_grad

        ## 更新原始变量  
        if len(flip_list) > 0:
            ## 对x_N进行翻转
            idx_flip_to_lower = flip_list[sol.sign[flip_list] == VarStatus.AT_UPPER_BOUND.value]
            idx_flip_to_upper = flip_list[sol.sign[flip_list] == VarStatus.AT_LOWER_BOUND.value]
            sol.x[idx_flip_to_lower] = problem.l[idx_flip_to_lower]
            sol.x[idx_flip_to_upper] = problem.u[idx_flip_to_upper]
            sol.sign[idx_flip_to_lower] = VarStatus.AT_LOWER_BOUND.value
            sol.sign[idx_flip_to_upper] = VarStatus.AT_UPPER_BOUND.value
            ## 根据翻转的x_N，更新x_B
            delta_x_flipped = np.zeros((basis.m,))
            delta_x_flipped[idx_flip_to_lower] = -problem.bounds_gap[idx_flip_to_lower]
            delta_x_flipped[idx_flip_to_upper] = problem.bounds_gap[idx_flip_to_upper]
            delta_b_flipped = basis.dot(delta_x_flipped,if_transpose=False)
            delta_xB = - basis.solve(delta_b_flipped,if_transpose=False)
            sol.x[basis.idxB] += delta_xB
            delta_xBI = delta_xB[idxI]
        else:
            delta_xBI = 0

        ## 然后，计算原始步长，并更新x_j和x_B
        alpha_primal = (-primal_gap - delta_xBI) / xB_grad[idxI]
        sol.x[basis.idxB] += alpha_primal * xB_grad
        sol.x[idxBI] = problem.l[idxBI] if bool_to_lower_bound else problem.u[idxBI]
        sol.sign[idxBI] = VarStatus.AT_LOWER_BOUND.value if bool_to_lower_bound else VarStatus.AT_UPPER_BOUND.value
        sol.x[idxNJ] += alpha_primal
        sol.sign[idxNJ] = VarStatus.OTHER.value ## 进入B
        
        ## 检查解的对偶可行性并及时进行shift操作
        if len(check_list) > 0:
            problem,sol = self._shift(problem,sol,check_list=check_list)

        ## 更新基
        basis.idxB[idxI] = idxNJ
        basis.idxN[idxJ] = idxBI
        basis.boolN[idxBI] = True
        basis.boolN[idxNJ] = False
        ## 更新PFI和DSE信息
        eta_vec = -xB_grad0 / xB_grad0[idxI]
        eta_vec[idxI] += 1 / xB_grad0[idxI]
        eta = (idxI,eta_vec)
        basis.lu_update(eta=eta)
        basis.update_DSE_weights(idxI,xB_grad0,tau,betaI)        
        sol.s[basis.idxB] = 0
        
        return SolveStatus.ONGOING,problem,sol,basis
        
    def _loop(self,problem,sol,basis):
        '''
        增加对REFACTOR、ROLLBACK状态的处理
        '''
        count = self.global_info.get('count',0)
        phase = self.global_info.get('phase',2)
        start_time = self.global_info.get('start_time',time.time())
        
        while True:
            if count % 50000 == 0 and count > 0:
                print('resetting the DSE weights!')
                basis.reset_DSE_weights() ## DSE更新
            if basis.eta_count % 20 == 0 and count > 0:
                basis.lu_factorize() ## LU分解
                
            status,problem,sol,basis = self._step(problem,sol,basis) ## 做一步迭代
            count += 1
            self.global_info['count'] = count
            header = '{} (P{})'.format(count,phase)

            ## 每隔一定迭代步数监控解的状态
            if ((count % 100 == 0 and count > 0) and (status == SolveStatus.ONGOING)) \
              or (status not in (SolveStatus.ONGOING,SolveStatus.REFACTOR,SolveStatus.ROLLBACK)):
                infeas_dict,status_str = problem.check_sol_status(sol)
                if count % 10000 == 0 and count > 0:
                    print('{}  {}'.format(header,status_str))

                if infeas_dict['unbnd']:
                    ## 基对偶不可行（存在原始变量无界），直接回滚
                    print('{}  WARN Bnd err.'.format(header))
                    status = SolveStatus.ROLLBACK
                elif infeas_dict['sign']:
                    ## 基原始不可行（原始变量与基不一致），尝试根据基重新计算解
                    print('{}  WARN Sign err.'.format(header))
                    status = SolveStatus.REFACTOR
                elif infeas_dict['dual']:
                    ## 解对偶不可行，尝试LU重分解
                    print('{}  WARN Dual inf.'.format(header))
                    status = SolveStatus.REFACTOR
                elif infeas_dict['cons']:
                    ## 线性约束不满足，尝试LU重分解
                    print('{}  WARN Con inf.'.format(header))
                    status = SolveStatus.REFACTOR
                else:
                    if status == SolveStatus.ONGOING:
                        ## 解的状态ok，加入回滚堆栈中
                        if count % 500 == 0:
                            self._save_recent(problem,sol,basis,count,phase)
                    else:
                        ## 进入结束状态，打印并退出
                        print('{}  ({}) {}'.format(header,status.name,status_str))
                        return status,problem,sol,basis
                
            ## 如果状态是LU重分解，进行重分解并分析后续方向
            if status == SolveStatus.REFACTOR:
                problem,sol,basis = self._refactorize(problem,sol,basis)
                infeas_dict,status_str = problem.check_sol_status(sol) ## 检查重分解后解的状态
                if infeas_dict['cons']:
                    ## 约束不满足，说明目前基的条件数过大，有必要进入回滚流程
                    print('{}  WARN Con inf after refactor. cond number of curr basis may be too large.'.format(header))
                    status = SolveStatus.ROLLBACK
                elif infeas_dict['dual'] or infeas_dict['sign']:
                    ## 解或基对偶不可行，重新进入第一阶段
                    print('{}  WARN Dual inf after refactor.'.format(header))
                    status = SolveStatus.PHASE1
                    phase = self.global_info['phase']
                else:
                    ## 解状态正常，继续迭代过程
                    status = SolveStatus.ONGOING

            ## 如果状态是回滚，则进入回滚流程
            if status == SolveStatus.ROLLBACK:
                print('{}  INFO rollback to a recent feasible solution.'.format(header))
                status,problem,sol,basis = self._rollback(problem,phase)

            ## 如果状态非继续迭代，abort
            if status != SolveStatus.ONGOING:
                problem.check_sol_status(sol,print_func=print,print_header=header)
                return status,problem,sol,basis

            ## 限制迭代时长和次数
            if time.time() - start_time > 1.8e4 or count > 1e6:
                print('out of time / iterations.')
                problem.check_sol_status(sol,print_func=print,print_header=header)
                return SolveStatus.OTHER,problem,sol,basis
    
    def _solve(self,problem,sol,basis):
        ## 保存迭代过程中的信息，用于进行求解流程控制
        self.global_info['c_raw'] = problem.c.copy()
        self.global_info['rollback_stack'] = [(0,1,basis.copy(),sol.copy())]
        
        ## 检查初始基/解的状态
        infeas_dict,status_str = problem.check_sol_status(sol)
        if infeas_dict['unbnd']:
            ## 基对偶不可行，进入第一阶段
            status = SolveStatus.PHASE1
        elif infeas_dict['cons']:
            ## 解线性约束不可行，意味着初始条件数就很大；目前的求解流程无法处理，直接退出
            ## 注：这需要放在基对偶可行判断的后面，因为如果有原始变量无界，由于数值原因，解的线性约束很容易不可行
            print('Cond num of the initial basis is too large. Abort.')
            status = SolveStatus.ERR
        else:
            ## 可直接进入第二阶段
            status = SolveStatus.PHASE2

        ## 进入循环，在两阶段中跳转
        iter_count = 0
        while ((status in (SolveStatus.PHASE1,SolveStatus.PHASE2)) and (iter_count < 100)):
            iter_count += 1
            if status == SolveStatus.PHASE2:
                ## 进入第二阶段，寻找最优基/解
                self.global_info['phase'] = 2
                status,problem,sol,basis = self._solve_phase_two(problem,sol,basis)
            elif status == SolveStatus.PHASE1:
                ## 进入第一阶段，寻找可行基/解
                self.global_info['phase'] = 1
                statusPhase1,problemPhase1,solPhase1,basis = self._solve_phase_one(problem,sol,basis)
                ## 用第一阶段所得的基更新原始问题的解
                problem.c = problemPhase1.c
                sol.sign = solPhase1.sign.copy()

                ## 对第一阶段求解结果进行分析
                if statusPhase1 == SolveStatus.DUAL_INFEAS:
                    ## 如果返回的状态是对偶不可行，则求解流程有问题，直接退出
                    status = SolveStatus.ERR
                    # status = SolveStatus.PHASE1
                elif statusPhase1 in (SolveStatus.OPT,SolveStatus.PRIMAL_INFEAS):
                    ## 如果返回的状态是最优或原始不可行，再次检查当前基是否满足对偶可行性（不存在原始变量无界）
                    bool_unbnd = problem.eval_unbnd(sol)
                    if not np.any(bool_unbnd):
                        ## 基对偶可行，进入第二阶段
                        status = SolveStatus.PHASE2
                    else:
                        ## 基对偶不可行，说明第一阶段求解结果有瑕疵；尝试直接调整基来恢复对偶可行性，但后续需要检查解的对偶可行性
                        bool_resolve_phase1 = False
                        idxs = np.where(bool_unbnd)[0]
                        for idx in idxs:
                            if problem.bound_type[idx] == BoundType.FREE.value:
                                ## free变量无法通过调整基来恢复对偶可行性；
                                ## 直接重新进入第一阶段
                                bool_resolve_phase1 = True
                                break
                            elif problem.bound_type[idx] == BoundType.UPPER_BOUNDED.value:
                                sol.sign[idx] = VarStatus.AT_UPPER_BOUND.value
                            elif problem.bound_type[idx] == BoundType.LOWER_BOUNDED.value:
                                sol.sign[idx] = VarStatus.AT_LOWER_BOUND.value
                        if bool_resolve_phase1:
                            ## 重新进入第一阶段；尝试进行随机扰动
                            problem = self._perturb(problem)
                            status = SolveStatus.PHASE1
                        else:
                            ## 根据调整后的基计算解，并检查其状态
                            sol = self._compute_sol_from_basis(problem,basis,sign=sol.sign)
                            infeas_dict,status_str = problem.check_sol_status(sol)
                            if infeas_dict['sign'] or infeas_dict['unbnd']:
                                ## 基对偶或原始不可行；
                                ## 调整基后一般不会进入这里；一旦进入了，直接退出求解流程
                                print('ERR Basis Primal/Dual Inf. Abort.')
                                status = SolveStatus.ERR
                            elif infeas_dict['dual']:
                                ## 重新进入第一阶段；尝试进行随机扰动
                                problem = self._perturb(problem)
                                status = SolveStatus.PHASE1
                            elif infeas_dict['primal']:
                                status = SolveStatus.PHASE2
                            else:
                                status = SolveStatus.OPT
                else:
                    ## 返回的状态不是标准结束状态（OPT、PRIMAL_INFEAS、DUAL_INFEAS），则直接复制状态并进行随机扰动
                    problem = self._perturb(problem)
                    status = statusPhase1
        return status,problem,sol,basis

下面通过测试来看一下新的DS实现的效果。由于引入了随机扰动，多次执行的结果可能不一样；为了评估的一致性，这里选取了一个固定的随机种子32。

In [10]:
ds_p = DualSimplexSolverPlus3()
test(ds_p.solve,50000,random_seed=32)

random seed = 32.

Problem name: 25FV47, size: ((821, 1571),10400).
[----Launch PuLP----]
[----Launch Solver----]
142 (P1)  (OPT) Obj Primal 1.7753e-12 Dual 2.2803e-15
615 (P2)  WARN pivot size -4.5910e-06.
2676 (P2)  (OPT) Obj Primal 5.5018e+03 Dual 5.5018e+03
[----Begin evaluation----]
PuLP Eval: con inf=6.7406e-04,var inf=0.0000e+00,obj=5.5018e+03. Status: 1.
Solver Eval: con inf=2.5077e-06,var inf=2.0133e-08,obj=5.5018e+03. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.464, Solver = 3.652.
Two solvers match: PuLP = 5.5018e+03, Solver = 5.5018e+03.

Problem name: 80BAU3B, size: ((2262, 9799),21002).
[----Launch PuLP----]
[----Launch Solver----]
779 (P1)  (OPT) Obj Primal -1.1340e-11 Dual 0.0000e+00
3758 (P2)  (OPT) Obj Primal 9.8722e+05 Dual 9.8722e+05
[----Begin evaluation----]
PuLP Eval: con inf=1.0557e-03,var inf=0.0000e+00,obj=9.8722e+05. Status: 1.
Solver Eval: con inf=6.0566e-08,var inf=2.4372e-10,obj=9.8722e+05. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.880, Solver = 


Problem name: CRE-A, size: ((3516, 4067),14987).
[----Launch PuLP----]
[----Launch Solver----]
3543 (P2)  (OPT) Obj Primal 2.3595e+07 Dual 2.3595e+07
[----Begin evaluation----]
PuLP Eval: con inf=4.9258e-05,var inf=5.4474e-11,obj=2.3595e+07. Status: 1.
Solver Eval: con inf=5.8027e-10,var inf=1.1012e-10,obj=2.3595e+07. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.333, Solver = 4.201.
Two solvers match: PuLP = 2.3595e+07, Solver = 2.3595e+07.

Problem name: CRE-C, size: ((3068, 3678),13244).
[----Launch PuLP----]
[----Launch Solver----]
3161 (P2)  (OPT) Obj Primal 2.5275e+07 Dual 2.5275e+07
[----Begin evaluation----]
PuLP Eval: con inf=7.4379e-05,var inf=0.0000e+00,obj=2.5275e+07. Status: 1.
Solver Eval: con inf=6.5770e-10,var inf=1.7670e-10,obj=2.5275e+07. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.331, Solver = 3.469.
Two solvers match: PuLP = 2.5275e+07, Solver = 2.5275e+07.

Problem name: CYCLE, size: ((1903, 2857),20720).
[----Launch PuLP----]
[----Launch Solver----]
550 (P


Problem name: FORPLAN, size: ((161, 422),4564).
[----Launch PuLP----]
[----Launch Solver----]
36 (P1)  (OPT) Obj Primal -1.9852e-11 Dual 0.0000e+00
275 (P2)  (OPT) Obj Primal -6.6422e+02 Dual -6.6422e+02
[----Begin evaluation----]
PuLP Eval: con inf=6.0633e-03,var inf=0.0000e+00,obj=-6.6422e+02. Status: 1.
Solver Eval: con inf=4.9643e-06,var inf=2.4401e-09,obj=-6.6422e+02. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.057, Solver = 0.170.
Two solvers match: PuLP = -6.6422e+02, Solver = -6.6422e+02.

Problem name: GANGES, size: ((1309, 1681),6912).
[----Launch PuLP----]
[----Launch Solver----]
1349 (P2)  (OPT) Obj Primal -1.0959e+05 Dual -1.0959e+05
[----Begin evaluation----]
PuLP Eval: con inf=2.1363e-02,var inf=2.3675e-12,obj=-1.0959e+05. Status: 1.
Solver Eval: con inf=7.0365e-10,var inf=1.5872e-15,obj=-1.0959e+05. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.122, Solver = 0.973.
Two solvers match: PuLP = -1.0959e+05, Solver = -1.0959e+05.

Problem name: GFRD-PNC, size: ((616, 

2336 (P2)  WARN pivot size 2.8059e-06.
2670 (P2)  (OPT) Obj Primal 1.4076e+07 Dual 1.4076e+07
[----Begin evaluation----]
PuLP Eval: con inf=4.1907e-03,var inf=1.1800e-04,obj=1.4076e+07. Status: 1.
Solver Eval: con inf=8.9906e-06,var inf=0.0000e+00,obj=1.4076e+07. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.335, Solver = 3.221.
Two solvers match: PuLP = 1.4076e+07, Solver = 1.4076e+07.

Problem name: PDS-02, size: ((2953, 7535),16390).
[----Launch PuLP----]
[----Launch Solver----]
2214 (P2)  (OPT) Obj Primal 2.8858e+10 Dual 2.8858e+10
[----Begin evaluation----]
PuLP Eval: con inf=0.0000e+00,var inf=0.0000e+00,obj=2.8858e+10. Status: 1.
Solver Eval: con inf=2.6420e-12,var inf=1.9146e-12,obj=2.8858e+10. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.623, Solver = 3.174.
Two solvers match: PuLP = 2.8858e+10, Solver = 2.8858e+10.

Problem name: PEROLD, size: ((625, 1376),6018).
[----Launch PuLP----]
[----Launch Solver----]
128 (P1)  (OPT) Obj Primal 9.9606e-15 Dual 0.0000e+00
208 (P2) 

8139 (P2)  INFO max shift size: 5.75e-06.
8200 (P2)  WARN Con inf.
8200 (P2)  WARN Dual inf after refactor.
8200 (P2)  Obj Primal -5.1565e+03 Dual -5.1565e+03  Primal Inf 5.4182e+05 (321)  Dual Inf 3.8311e+02 (621)
8369 (P1)  WARN pivot size -6.8797e-07.
8449 (P1)  WARN pivot size 7.7937e-07.
9099 (P1)  (PRIMAL_INFEAS) Obj Primal 2.0983e-05 Dual 2.0845e-05  Primal Inf 3.7863e+00 (210)
9099 (P1)  INFO max perturb size 1.00e-05.
9235 (P1)  WARN pivot size 3.1280e-07.
9545 (P1)  (PRIMAL_INFEAS) Obj Primal -9.7509e-12 Dual 0.0000e+00  Primal Inf 2.0077e-05 (5)
9556 (P2)  INFO max shift size: 1.58e-06.
9561 (P2)  INFO max shift size: 6.92e-04.
9580 (P2)  INFO max shift size: 1.49e-04.
9581 (P2)  INFO max shift size: 2.13e-04.
9589 (P2)  INFO max shift size: 1.19e-03.
9600 (P2)  WARN Dual inf.
9600 (P2)  WARN Dual inf after refactor.
9600 (P2)  Obj Primal -9.9524e+03 Dual -9.9524e+03  Primal Inf 2.5837e+10 (309)  Dual Inf 6.3149e+00 (370)
10000 (P1)  Obj Primal 9.9911e-06 Dual 1.0028e-05  Pr

17636 (P1)  WARN pivot size 9.4230e-06.
17665 (P1)  WARN pivot size 1.5073e-07.
17723 (P1)  (PRIMAL_INFEAS) Obj Primal -6.7586e-08 Dual 0.0000e+00  Primal Inf 7.6821e-03 (96)
17800 (P2)  WARN Dual inf.
17800 (P2)  WARN Con inf after refactor. cond number of curr basis may be too large.
17800 (P2)  INFO rollback to a recent feasible solution.
17800 (P2)  INFO rollback to iter 17500 (P1).
17800 (P2)  INFO max perturb size 1.00e-05.
17800 (P2)  Obj Primal -2.1072e+00 Dual -3.3643e+14  Primal Inf 3.2000e+17 (324)  Dual Inf 3.3643e-02 (31)  Con Inf 6.5603e+05 (691)  Bnd err 32  Sign err 198
17822 (P1)  WARN pivot size -6.3180e-06.
17900 (P1)  WARN Con inf.
18481 (P1)  WARN pivot size 1.4565e-06.
18669 (P1)  WARN pivot size 1.2664e-06.
18725 (P1)  WARN pivot size 1.1952e-06.
18759 (P1)  WARN pivot size -9.6646e-06.
18800 (P1)  WARN Con inf.
18898 (P1)  WARN pivot size 7.4701e-06.
18938 (P1)  INFO rollback to a recent feasible solution.
18938 (P1)  INFO rollback to iter 18500 (P1).
18938 (P1)

1628 (P2)  WARN pivot size 5.8291e-06.
1658 (P2)  WARN pivot size 1.6433e-06.
1918 (P2)  WARN pivot size 1.6361e-06.
3843 (P2)  WARN pivot size 1.6973e-06.
3939 (P2)  WARN pivot size -3.8174e-06.
3955 (P2)  WARN pivot size 7.5177e-06.
3971 (P2)  WARN pivot size 2.9504e-06.
4106 (P2)  WARN pivot size 1.9358e-06.
4135 (P2)  WARN pivot size 5.7662e-07.
4153 (P2)  WARN pivot size 5.5517e-06.
4157 (P2)  WARN pivot size 3.5209e-06.
4182 (P2)  WARN pivot size -7.5787e-06.
4230 (P2)  (OPT) Obj Primal -5.5567e+02 Dual -5.5567e+02
[----Begin evaluation----]
PuLP Eval: con inf=1.0090e-03,var inf=3.2044e-05,obj=-5.5749e+02. Status: 1.
Solver Eval: con inf=1.9398e-06,var inf=2.9718e-09,obj=-5.5747e+02. Status: SolveStatus.OPT.
Elapsed time: PuLP = 1.998, Solver = 11.234.
Two solvers match: PuLP = -5.5749e+02, Solver = -5.5747e+02.

Problem name: PILOT4, size: ((410, 1000),5141).
[----Launch PuLP----]
[----Launch Solver----]
110 (P1)  (OPT) Obj Primal 1.4488e-14 Dual 0.0000e+00
730 (P2)  (OPT) Obj P

281 (P1)  (OPT) Obj Primal -4.3758e-13 Dual 0.0000e+00
993 (P2)  (OPT) Obj Primal 3.6660e+04 Dual 3.6660e+04
[----Begin evaluation----]
PuLP Eval: con inf=1.5460e-03,var inf=2.4365e-08,obj=3.6660e+04. Status: 1.
Solver Eval: con inf=2.8298e-08,var inf=1.5048e-10,obj=3.6660e+04. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.091, Solver = 0.674.
Two solvers match: PuLP = 3.6660e+04, Solver = 3.6660e+04.

Problem name: SCFXM3, size: ((990, 1371),7777).
[----Launch PuLP----]
[----Launch Solver----]
391 (P1)  (OPT) Obj Primal -1.6919e-12 Dual 0.0000e+00
1464 (P2)  (OPT) Obj Primal 5.4901e+04 Dual 5.4901e+04
[----Begin evaluation----]
PuLP Eval: con inf=1.5061e-03,var inf=2.4806e-08,obj=5.4901e+04. Status: 1.
Solver Eval: con inf=4.5944e-08,var inf=1.6047e-10,obj=5.4901e+04. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.153, Solver = 1.135.
Two solvers match: PuLP = 5.4901e+04, Solver = 5.4901e+04.

Problem name: SCORPION, size: ((388, 358),1426).
[----Launch PuLP----]
[----Launch Solver


Problem name: SHIP12L, size: ((1151, 5427),16170).
[----Launch PuLP----]
[----Launch Solver----]
1203 (P2)  (OPT) Obj Primal 1.4702e+06 Dual 1.4702e+06
[----Begin evaluation----]
PuLP Eval: con inf=4.0179e-06,var inf=1.4003e-11,obj=1.4702e+06. Status: 1.
Solver Eval: con inf=1.4548e-12,var inf=7.6624e-14,obj=1.4702e+06. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.283, Solver = 0.819.
Two solvers match: PuLP = 1.4702e+06, Solver = 1.4702e+06.

Problem name: SHIP12S, size: ((1151, 2763),8178).
[----Launch PuLP----]
[----Launch Solver----]
1132 (P2)  (OPT) Obj Primal 1.4892e+06 Dual 1.4892e+06
[----Begin evaluation----]
PuLP Eval: con inf=4.2039e-06,var inf=9.0000e-12,obj=1.4892e+06. Status: 1.
Solver Eval: con inf=1.6349e-12,var inf=8.1528e-14,obj=1.4892e+06. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.163, Solver = 0.721.
Two solvers match: PuLP = 1.4892e+06, Solver = 1.4892e+06.

Problem name: SIERRA, size: ((1227, 2036),7302).
[----Launch PuLP----]
[----Launch Solver----]
621

上面的测试结果显示，新的DS实现基本可以正确求解所有测试问题；但是，求解效率相比PuLP还是有很大差距。在下一篇blog/notebook中，我将进一步引入一些预处理手段来提高求解效率。